# Import bibliotek

In [34]:
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from operator import itemgetter
import time
from xgboost import XGBRegressor

# Oszacowanie modelu XGBoost

In [65]:
features=[]
df=pd.DataFrame()
def CVTestXGB(nFolds = 5, randomState=2024, debug=False, features=features, *args, **kwargs):
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)
    # Listy z wynikami
    testResults = []
    trainResults = []
    predictions = []
    indices = []
    # Pętla walidująca model na kolejnych foldach
    for train, test in kf.split(df.index.values):
        # Przygotowanie estymatora
        clf = XGBRegressor(*args, **kwargs)
        # Trenowanie modelu
        clf.fit(df.iloc[train][features], df.iloc[train][target])
        # Przygotowanie prognoz dla zbioru treningowego i testowego
        predsTrain = clf.predict(df.iloc[train][features])
        preds = clf.predict(df.iloc[test][features])
        # Zachowajmy informacje o predykcjach dla tego foldu
        predictions.append(preds.tolist().copy())
        # Razem z indeksami w oryginalnym data frame
        indices.append(df.iloc[test].index.tolist().copy())
        # Policzenie RMSE dla foldów
        trainScore = metrics.mean_squared_error(df.iloc[train][target], predsTrain)**0.5
        testScore = metrics.mean_squared_error(df.iloc[test][target], preds)**0.5
        # Zapisanie wyników dla foldów
        trainResults.append(trainScore)
        testResults.append(testScore)
        # Informowanie o każdym foldzie razem z wynikami treningowymi możemy opcjonalnie wyświetlać w trakcie
        if debug:
            print("Train RMSE:", trainScore,
                  "Valid RMSE:", testScore)
        
    return trainResults, testResults, predictions, indices

Zobaczymy czy w tym przypadku feature engineering poprawił jakość predykcyjną:

Model bez feature engineeringu:

In [46]:
df=pd.read_csv("data_eda.csv")
df=df.drop(columns=['Unnamed: 0'])
features=df.columns.tolist()
features.remove('stars')
target='stars'

In [47]:
trainResults, testResults, predictions, indices = CVTestXGB(features=features)
print(np.mean(testResults))

0.19323604798601268


Model z interakcjami:

In [48]:
df=pd.read_csv("data_add.csv")
df=df.drop(columns=['Unnamed: 0'])
features=df.columns.tolist()
features.remove('stars')
target='stars'

In [49]:
trainResults, testResults, predictions, indices = CVTestXGB(features=features)
print(np.mean(testResults))

0.19442793207844053


Model z interakcjami i transformacją zmiennych:

In [50]:
df=pd.read_csv("data_fe.csv")
df=df.drop(columns=['Unnamed: 0'])
features=df.columns.tolist()
features.remove('stars')
target='stars'

In [51]:
trainResults, testResults, predictions, indices = CVTestXGB(features=features)
print(np.mean(testResults))

0.20262947349887567


Widzimy, że dla takiego silnego modelu najlepszy jest zbiór danych bez interakcji i transformacji zmiennych.

In [68]:
df=pd.read_csv("data_eda.csv")
df=df.drop(columns=['Unnamed: 0'])
features=df.columns.tolist()
features.remove('stars')
target='stars'

Moc predykcji jest już lepsza od poprzedni modeli, ale postramy się ją poprawić poprzez tuning parametrów

# Tuning hiperparametrów XGBoost

1. Głębokość drzewa

In [77]:
for x in range(3,10):
    trainResults, testResults, predictions, indices = CVTestXGB(features=features, max_depth=x)
    print(x,np.mean(testResults))

3 0.19411233131329214
4 0.1913640556683877
5 0.1919094913508645
6 0.19323604798601268
7 0.19622217895826793
8 0.19719521429921177
9 0.20181620462247313


Najlepszy wynik dla głębokości drzewa równej 4.

2. Udziału wierszy i kolumn w odróżnieniu od głębokości drzewa poszukamy poprzez random search

In [81]:
import random
results=[]
paramList = []
for x in range(100):
    params = (random.randint(3,10),random.randint(3,10),random.randint(3,10))
    trainResults, testResults, predictions, indices = CVTestXGB(
        features=features,
        max_depth=4,
        subsample = params[0]/10,
        colsample_bytree = params[1]/10,
        colsample_bylevel=params[2]/10)
    results.append(np.mean(testResults))
    paramList.append(params)

In [82]:
maxRes = []
for i in range(1, len(paramList)):
    maxRes.append(max(results[0:i]))
imp = list(zip(results, paramList))
imp.sort(reverse=False)
for row in imp[0:10]:
    print(row)

(0.1892602742829848, (9, 4, 7))
(0.19061760717369958, (9, 9, 5))
(0.19061760717369958, (9, 9, 5))
(0.19075002430848503, (9, 8, 5))
(0.19086323736510097, (9, 4, 8))
(0.19098045331501562, (9, 6, 6))
(0.19099192182920746, (8, 9, 10))
(0.19099484753556367, (10, 7, 6))
(0.19104719268035805, (9, 5, 10))
(0.1910946910302517, (7, 8, 5))


Najlepszy wynik dla udziału wierszy i kolumn równego: 0.9, 0.4, 0.7

In [84]:
trainResults, testResults, predictions, indices = CVTestXGB(
        features=features,
        max_depth=4,
        subsample = 0.9,
        colsample_bytree = 0.4,
        colsample_bylevel=0.7, debug=True)
print(np.mean(testResults))

Train RMSE: 0.14124132729533329 Valid RMSE: 0.1887327791361306
Train RMSE: 0.1400144649772968 Valid RMSE: 0.1890066999855199
Train RMSE: 0.13982620692436126 Valid RMSE: 0.1891888358518881
Train RMSE: 0.14148343773361183 Valid RMSE: 0.1915223454087249
Train RMSE: 0.14319308287804947 Valid RMSE: 0.18785071103266052
0.1892602742829848


Widzimy, że jednak nadal wynik na zbiorze treningowym znacząco lepszy niż testowym, więc dostosujemy parametry regularyzacji.

# Regularyzacja

Tutaj poszukamy parametru lambda (regularyzacja L2).

In [92]:
for x in [0.01, 0.1, 0.5, 1, 2, 4, 6, 8, 10, 15]:
    trainResults, testResults, predictions, indices = CVTestXGB(
        features=features,
        max_depth=4,
        subsample = 0.9,
        colsample_bytree = 0.4,
        colsample_bylevel=0.7, reg_lambda=x)
    print(x,np.mean(testResults))

0.01 0.1912946145324798
0.1 0.19174927656118912
0.5 0.19162481890130426
1 0.1892602742829848
2 0.19004836047688012
4 0.18921250694031722
6 0.19108452370704412
8 0.1895263013482507
10 0.19042703852380521
15 0.19010262656740093


In [94]:
for x in range(30,50):
    trainResults, testResults, predictions, indices = CVTestXGB(
        features=features,
        max_depth=4,
        subsample = 0.9,
        colsample_bytree = 0.4,
        colsample_bylevel=0.7, reg_lambda=x/10)
    print(x/10,np.mean(testResults))

3.0 0.1895081754802272
3.1 0.18973658524204615
3.2 0.19005102837290858
3.3 0.18927938881848774
3.4 0.1891394298795249
3.5 0.18966566734930765
3.6 0.18880099567858424
3.7 0.1908712470367836
3.8 0.19004917839514043
3.9 0.18967418596374536
4.0 0.18921250694031722
4.1 0.1900763491950995
4.2 0.1895148397283905
4.3 0.19099688087836503
4.4 0.19031931539356425
4.5 0.1903055708213813
4.6 0.19041159128271348
4.7 0.19053302597083835
4.8 0.19030106582780731
4.9 0.18980898409241231


Uzyskaliśmy nieznaczną poprawę przy lambda=3.6

In [96]:
trainResults, testResults, predictions, indices = CVTestXGB(
        features=features,
        max_depth=4,
        subsample = 0.9,
        colsample_bytree = 0.4,
        colsample_bylevel=0.7, reg_lambda=3.6)
print(np.mean(testResults))

#Zapisujemy model
modelXGBoost = {
    "name":"XGBoost",
    "trainResults":trainResults.copy(),
    "testResults":testResults.copy(),
    "predictions":predictions.copy(),
    "indices":indices.copy(),
}

import pickle

# Otwieramy plik do zapisu binarnego z wykorzystenim with
with open("model_XGBoost.p", "wb") as fp:
    # Zapisujemy obiekt do wskaźnika pliku
    pickle.dump(modelXGBoost, fp)

0.18880099567858424


Ostateczny wynik dla modelu XGBoost to *0.18880*